<a href="https://colab.research.google.com/github/talitanog/mpvsp12024/blob/main/prevsituacaoevento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Classificador de Contestação de Eventos de Mudança de Estado Operativo de Unidades Geradoras**

**Contexto:** Temos um conjunto de dados que se chamam eventos. Eventos são as alterações de estados operativos de unidades geradoras de usinas térmicas, hidraulicas e nucleares. Cada mudança de disponibilidade da unidade geradora tem um evento registrado. Esse evento passa por uma primeira análise e depois é disponibilizado para o Agente Proprietário do equipamento. O Agente proprietário poderá discordar da primeira análise e esse evento então receberá um status diferente daquele inicial.
Queremos identificar: dado um evento que possui informações de usina, unidade geradora, classificadores que indicam seu estado operativo, disponibilidade e data hora a previsão de contestação pelo agente. Ou seja, se o evento terá a classificação como "NÃO ACORDADO" após a primeira análise.


**Estrtura:** Divisão do notebook:
1. Importação das bibliotecas
2. Configuração para não exibir warnings
3. Acessando do dataset que será utilizado
4. Tratamento dos dados
5. Definição de features e target
6. Divisão em Treino e Teste
7. Treinamento do modelo machine learning
8. Aplicação de Hiperparâmetros
9. Finalização do Modelo
10. Utilização do Modelo escolhido no treinamento
11. Simulação em dados não vistos
12. Conclusão



# **CHECKLIST DEFINIÇÃO DO PROBLEMA**

**Premissas e Hipóteses:**
Não existe premissa, a principal hipótese é que determinados agentes, proprietários dos equipamentos apurados, através dos eventos do nosso dataset, irão sempre contestar um evento que possua determinadas características ou estiver em determinada sequência.
Por isso, a tentativa de identificar uma regra através do modelo.

**Restrições:**
Um evento ele pode ser contestado em um primeiro momento e logo depois a equipe que faz a primeira análise pode concordar com essa contestação. Quando isso ocorre o evento fica com a classficação de "Acordado", como se ele nunca tivesse passado por uma contestação.
Não vamos tratar essa mudança de status intermediária nesse trabalho.

**Descreva o dataset:**
usina_id: ID da Usina
usina_nome: Nome da Usina
unidade_geradora_id: ID da Unidade Geradora
unidade_geradora_nome: Nome da Unidade Geradora
evento_data_ocorrencia: Evento Data Ocorrência (data/hora em que houve mudança de disponibilidade da unidade geradora)
evento_eo: Estado Operativo da Unidade Geradora (Ex: Ligada/Desligada)
evento_co: Condição Operativo da Unidade Geradora (Ex: Normal/Restrição)
evento_or: Origem da Condição Operativa da Unidade Geradora (Ex: Causa interna/externa/falta de combustível)
evento_disponibilidade: Disponibilidade apresentada pela unidade geradora (Ex: 100MW)
evento_status: Status do Evento
evento_situacao: Acordado / Não Acordado

# **1. Importando as bibliotecas necessárias para executar o notebook**

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms # para tratamento de missings
from joblib import Parallel, delayed
from matplotlib import cm
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.svm import SVR
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# **2. Configuração para não exibir warnings**

In [ ]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# **3. Acessando dataset que será utilizado**

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

In [ ]:
# Lê o arquivo Excel
eventos = pd.read_excel('/content/Cópia de eventos_talita_mba_v8.xlsx')

# Exibe os primeiros registros do DataFrame
print(eventos.head())

# **4. Tratamento dos dados**

# CHECKLIST MODELAGEM E TREINAMENTO

**Selecione os algoritmos mais indicados para o problema e dataset escolhidos, justificando as suas escolhas.**
Abaixo justifico a retirada das colunas menos importantes. Na conclusão explico o porque mantive as colunas que foram utilizadas no treino.

**Há algum ajuste inicial para os hiperparâmetros?**
Sim, foi realizado após o primeiro treinamento.

**O modelo foi devidamente treinado? Foi observado problema de underfitting?**
Sim, Não identifiquei underfitting.

**É possível otimizar os hiperparâmetros de algum dos modelos? Se sim, faça-o, justificando todas as escolhas.**
Foi realizado, porém não apresentou grande variação. está detalhado no decorrer do notebook.

**Há algum método avançado ou mais complexo que possa ser avaliado?**
Sim, acredito que novas variáveis podem ser adicionadas, inclusive status intermediários dos eventos.

**Posso criar um comitê de modelos diferentes para o problema (ensembles)?**
Sim, foi realizado no meu trabaho de classificação, explicado também na conclusão. Porém, não utilizado. Realizei o Bagging, Random Forest, Boosting, etc.


In [ ]:
# Converte a coluna 'evento_data_ocorrencia' para o tipo datetime
eventos['evento_data_ocorrencia'] = pd.to_datetime(eventos['evento_data_ocorrencia'])

In [ ]:
# Separa coluna evento_data_ocorrência em 05 colunas para cada tipo de informação
eventos['ano'] = eventos['evento_data_ocorrencia'].dt.year
eventos['mes'] = eventos['evento_data_ocorrencia'].dt.month
eventos['dia'] = eventos['evento_data_ocorrencia'].dt.day
eventos['hora'] = eventos['evento_data_ocorrencia'].dt.hour
eventos['minuto'] = eventos['evento_data_ocorrencia'].dt.minute

In [ ]:
# Exclusão de colunas desnecessárias para a análise
eventos = eventos.drop(['usina_nome','evento_data_ocorrencia', 'unidade_geradora_nome', 'evento_status'], axis=1)
print (eventos.head())

Razão para exclusão dos campos:

1. usina_nome: Mantive a coluna de ids da usina
2. evento_data_ocorrencia: Essa coluna foi traduzida em 05 colunas conforme comando acima.
3. unidade_geradora_nome: Mantive a coluna de ids das unidades geradoras
4. evento_status: Essa coluna foi traduzida na coluna target


In [ ]:
# Ordenação das colunas
nova_ordem_colunas = ['usina_id','unidade_geradora_id', 'evento_eo', 'evento_co', 'evento_or', 'evento_disponibilidade', 'ano','mes','dia','hora','minuto','evento_situacao']

# Reordenar as colunas do DataFrame
eventos = eventos[nova_ordem_colunas]

print (eventos.head())

Para converter as variáveis categóricas evento_eo, evento_co e evento_or utilizei o one hot encoding.
Significado de cada um desses atributos:
evento_eo: Estado Operativo, em resumo essa informação define se o evento representa que a unidade geradora está ligada ou desligada.
evento_co: Condição Operativa, resumidamente define em que condições a unidade geradora está ligada, as condições podem ser Normal ou Restrição.
evento_or: Origem, resumidamente diz qual a origem do desligamento ou da restrição que a unidade geradora está apresentando.

In [ ]:
# Realizar One-Hot Encoding para as colunas 'evento_eo', 'evento_co' e 'evento_or'
eventos_encoded = pd.get_dummies(eventos, columns=['evento_eo', 'evento_co', 'evento_or'])

# Exibir o DataFrame com as colunas codificadas
print(eventos_encoded)

Para a transformação da coluna target em variáveis numéricas utilizei o Label encoder.
A coluna evento_situacao foi transformada em evento_situacao_num.


In [ ]:
# Crie uma instância do LabelEncoder
label_encoder = LabelEncoder()

# Ajuste o LabelEncoder aos valores da coluna target
label_encoder.fit(eventos_encoded['evento_situacao'])

# Transforme os valores da coluna target em valores numéricos
evento_situacao_num = label_encoder.transform(eventos_encoded['evento_situacao'])

# Adicione a coluna transformada de volta ao DataFrame
eventos_encoded['evento_situacao_num'] = evento_situacao_num

# Agora a coluna 'evento_situacao_num' estará presente no DataFrame eventos_encoded
evento_situacao_num

Apenas dois valores foram determinados para essa atividade. "Acordado" através do LabelEncoder virou 0 e 1 "Não Acordado"

Abaixo confirmei todas as colunas do dataset

In [ ]:
for coluna in eventos_encoded.columns:
  print(coluna)

In [ ]:
# Verificar os valores de evento_situacao e evento_situacao_num
print(eventos_encoded['evento_situacao'])
print(eventos_encoded['evento_situacao_num'])

# **5. Definição de target**

In [ ]:
# Separação de colunas e definição target

# Remover a coluna original após a codificação
eventos_encoded = eventos_encoded.drop(columns=['evento_situacao'])

# Definição do Target
coluna_target = 'evento_situacao_num'

# Definição das Features
features = eventos_encoded.drop(columns=[coluna_target])


# **6. Divisão de Treino e teste**

In [ ]:
# Divisão dos Dados em treino e teste

X_train, X_test, y_train, y_test = train_test_split(features, eventos_encoded[coluna_target], test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

# **7. Treinamento do modelo machine learning**

In [ ]:
# Definindo uma seed global para o gerador de números aleatórios
np.random.seed(42)

# Definir o número de folds para a validação cruzada
kfold = KFold(n_splits=10)

# Listas para armazenar os modelos, os resultados e os nomes dos modelos
models = []
results = {'LR': [], 'Ridge': [], 'Lasso': [], 'CART': []}
names = []

# Preparando os modelos e adicionando-os em uma lista
models.append(('LR', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('Lasso', Lasso()))
models.append(('CART', DecisionTreeRegressor()))

# Avaliando um modelo por vez
for name, model in models:
    scoring = {'neg_mean_squared_error': 'neg_mean_squared_error', 'r2': make_scorer(r2_score), 'mae': make_scorer(mean_absolute_error)}
    cv_results = cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
    results[name] = cv_results
    names.append(name)
    # Imprime as métricas dos 10 resultados da validação cruzada
    msg = "%s: MSE %0.2f (%0.2f) - RMSE %0.2f, R² %0.2f, MAE %0.2f" % (
        name, abs(cv_results['test_neg_mean_squared_error'].mean()),
        cv_results['test_neg_mean_squared_error'].std(),
        np.sqrt(abs(cv_results['test_neg_mean_squared_error'].mean())),
        cv_results['test_r2'].mean(),
        cv_results['test_mae'].mean())
    print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure()
fig.suptitle('Comparação do MSE, R² e MAE dos Modelos')
ax = fig.add_subplot(111)
plt.boxplot([results[name]['test_neg_mean_squared_error'] for name in names])
ax.set_xticklabels(names)
plt.show()


Os modelos LR e Ridge apresentaram resultados semelhantes, porém a Árvore de Decisão apresentou o melhor resultado

## 7.1. Comparação entre modelos

In [ ]:
# Boxplot de comparação dos modelos
fig, axes = plt.subplots(2, 1, figsize=(10, 8))
fig.suptitle('Comparação do MSE, R² e MAE dos Modelos')

# Boxplot para MSE
ax1 = axes[0]
ax1.boxplot([results[name]['test_neg_mean_squared_error'] for name in names])
ax1.set_xticklabels(names)
ax1.set_title('Comparação do MSE dos Modelos')

# Gráfico de barras para R²
ax2 = axes[1]
ax2.bar(names, [results[name]['test_r2'].mean() for name in names])
ax2.set_title('Comparação do R² dos Modelos')

plt.tight_layout()
plt.show()

A árvore de decisão tem um desempenho muito melhor, observando o MSE e RMSE que são muito baixos, indicando uma boa precisão, o R² próximo a 1 sugere que o modelo explica quase toda a variabilidade nos dados e o MAE é praticamente zero, o que indica um ajuste muito preciso aos dados.

# **8. Hiperparâmetros**

In [ ]:
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=DecisionTreeRegressor(),
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           cv=10,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)


## 8.1. Apresentação dos resultados

In [ ]:
best_params = grid_search.best_params_
print("Melhores parâmetros:", best_params)


In [ ]:
best_model = grid_search.best_estimator_
print("Melhor modelo:", best_model)

In [ ]:
cv_results = grid_search.cv_results_
print("Resultados da validação cruzada:", cv_results)

## 8.2. Resultados após a aplicação de Hiperparâmetros

In [ ]:
# Previsão do modelo
y_pred = best_model.predict(X_test)

# Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Coeficiente de Determinação (R²)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de Determinação (R²):", r2)

# Erro Absoluto Médio (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("Erro Absoluto Médio (MAE):", mae)

# Raiz do Erro Quadrático Médio (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Raiz do Erro Quadrático Médio (RMSE):", rmse)

Não tivemos grandes melhorias em aplicar o hiperparâmetro.

# **9. Finalização do modelo**

In [ ]:
np.random.seed(42) # Definindo uma semente global para este bloco

# Listas para armazenar os pipelines e os resultados para todas as visões do dataset
pipelines = []
results = []
names = []

# Criando os elementos do pipeline

# Modelos que serão utilizados
linear_reg = ('LR', LinearRegression())
ridge = ('Ridge', Ridge())
lasso = ('Lasso', Lasso())
decision_tree = ('CART', DecisionTreeClassifier())

# Transformações que serão utilizadas
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())

# Montando os pipelines

# Dataset original
pipelines.append(('LR-orig', Pipeline([linear_reg])))
pipelines.append(('Ridge-orig', Pipeline([ridge])))
pipelines.append(('Lasso-orig', Pipeline([lasso])))
pipelines.append(('CART-orig', Pipeline([decision_tree])))

# Dataset Padronizado
pipelines.append(('LR-padr', Pipeline([standard_scaler, linear_reg])))
pipelines.append(('Ridge-padr', Pipeline([standard_scaler, ridge])))
pipelines.append(('Lasso-padr', Pipeline([standard_scaler, lasso])))
pipelines.append(('CART-padr', Pipeline([standard_scaler, decision_tree])))

# Dataset Normalizado
pipelines.append(('LR-norm', Pipeline([min_max_scaler, linear_reg])))
pipelines.append(('Ridge-norm', Pipeline([min_max_scaler, ridge])))
pipelines.append(('Lasso-norm', Pipeline([min_max_scaler, lasso])))
pipelines.append(('CART-norm', Pipeline([min_max_scaler, decision_tree])))

# Executando os pipelines
for name, model in pipelines:
    cv_results = cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %.3f (%.3f)" % (name, cv_results['test_neg_mean_squared_error'].mean(), cv_results['test_neg_mean_squared_error'].std()) # Formatando para 3 casas decimais
    print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(25,6))
fig.suptitle('Comparação dos Modelos - Dataset orginal, padronizado e normalizado')
ax = fig.add_subplot(111)
plt.boxplot([result['test_neg_mean_squared_error'] for result in results])
ax.set_xticklabels(names, rotation=90)
plt.show()

Ajuste do negativo do MSE.
Todos os modelos de regressão do Ridge e LR tiveram desempenho semelhante.
Os modelos do Lasso tiveram um desempenho pior comparados com os outros modelos.
O modelo que teve melhor desempenho foi o DecisionTree
.


## 9.1. Matriz de Confusão

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(conf_matrix)

O modelo parece estar tendo um bom desempenho pelo resultado da matrix. Os valores de verdadeiro positivo e verdadeiro negativo são altos.

## 9.2. Preparação do modelo escolhido usando a pipeline

In [ ]:
# Definindo uma semente global para este bloco
np.random.seed(42)

# Preparação do modelo usando Pipeline
model_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Padronização dos dados
    ('model', DecisionTreeClassifier())  # Modelo de regressão linear
])

# Treinamento do modelo com o conjunto de treinamento
model_pipeline.fit(X_train, y_train)

# Previsões no conjunto de teste
predictions = model_pipeline.predict(X_test)

# Calculando as métricas de avaliação
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)

print("Mean Squared Error (MSE):", mse)
print("Coeficiente de Determinação (R²):", r2)
print("Erro Absoluto Médio (MAE):", mae)

Modelo de regressão está performando bem. O MSE, R² e MAE apresentam valores baixos.

In [ ]:
# Preparação do modelo com TODO o dataset
scaler = StandardScaler().fit(eventos_encoded) # ajuste do scaler com TODO o dataset
rescaledX = scaler.transform(eventos_encoded) # aplicação da padronização com TODO o dataset
model.fit(rescaledX, eventos_encoded['evento_situacao_num'])

# **10. Utilizando modelo Decision Tree em dados novos**

## 10.1. Importando arquivo

In [ ]:
# Lê o arquivo Excel de trabalho [Dados não conhecidos]
eventos_novos = pd.read_excel('/content/trabalhov2.xlsx')

# Exibe os primeiros registros do DataFrame
print(eventos_novos.head())

## 10.2. Tratamento dos dados para ser lido pelo modelo

In [ ]:
# Converte a coluna 'evento_data_ocorrencia' para o tipo datetime
eventos_novos['evento_data_ocorrencia'] = pd.to_datetime(eventos_novos['evento_data_ocorrencia'])

In [ ]:
# Separa coluna evento_data_ocorrência em 05 colunas para cada tipo de informação
eventos_novos['ano'] = eventos_novos['evento_data_ocorrencia'].dt.year
eventos_novos['mes'] = eventos_novos['evento_data_ocorrencia'].dt.month
eventos_novos['dia'] = eventos_novos['evento_data_ocorrencia'].dt.day
eventos_novos['hora'] = eventos_novos['evento_data_ocorrencia'].dt.hour
eventos_novos['minuto'] = eventos_novos['evento_data_ocorrencia'].dt.minute

In [ ]:
# Exclusão de colunas desnecessárias para a análise
eventos_novos = eventos_novos.drop(['usina_nome','evento_data_ocorrencia', 'unidade_geradora_nome'], axis=1)
print (eventos_novos.head())

In [ ]:
# Ordenação das colunas
ordenacao_colunas = ['usina_id','unidade_geradora_id', 'evento_eo', 'evento_co', 'evento_or', 'evento_disponibilidade', 'ano','mes','dia','hora','minuto']

# Reordenar as colunas do DataFrame
eventos_novos = eventos_novos[ordenacao_colunas]

print (eventos_novos.head())

In [ ]:
# Realizar One-Hot Encoding para as colunas 'evento_eo', 'evento_co' e 'evento_or'
eventos_novos_encoded = pd.get_dummies(eventos_novos, columns=['evento_eo', 'evento_co', 'evento_or'])

# Exibir o DataFrame com as colunas codificadas
print(eventos_novos_encoded)

## 10.3. Confirmação se as features são as mesmas

In [ ]:
# Verificar se os nomes das colunas nos dados de treinamento e nos dados de eventos novos são os mesmos
columns_X_train = set(X_train.columns)
columns_eventos_novos_encoded = set(eventos_novos_encoded.columns)

# Verificar se todas as colunas de X_train estão presentes em eventos_novos_encoded
missing_columns_in_eventos_novos_encoded = columns_X_train - columns_eventos_novos_encoded

# Verificar se todas as colunas de eventos_novos_encoded estão presentes em X_train
missing_columns_in_X_train = columns_eventos_novos_encoded - columns_X_train

missing_columns_in_eventos_novos_encoded, missing_columns_in_X_train

In [ ]:
# Lista das colunas presentes em X_train, mas ausentes em eventos_novos_encoded
colunas_ausentes = list(missing_columns_in_eventos_novos_encoded)

# Adicionar as colunas ausentes aos dados de eventos_novos_encoded e preenchê-las com "falso"
for col in colunas_ausentes:
    eventos_novos_encoded[col] = False

# Verificar se todas as colunas de X_train agora estão presentes em eventos_novos_encoded
missing_columns_in_eventos_novos_encoded = set(X_train.columns) - set(eventos_novos_encoded.columns)

# Exibir as colunas ausentes após a adição
print("Colunas ausentes em eventos_novos_encoded após a adição:", missing_columns_in_eventos_novos_encoded)

In [ ]:
# Verificar os recursos nos dados de entrada e treinamento
features_input = eventos_novos_encoded.columns.tolist()
features_training = X_train.columns.tolist()

# Comparar os conjuntos de recursos
if set(features_input) == set(features_training):
    print("Os conjuntos de recursos são iguais.")
else:
    print("Os conjuntos de recursos são diferentes.")

    # Identificar as diferenças
    features_only_in_input = set(features_input) - set(features_training)
    features_only_in_training = set(features_training) - set(features_input)

    # Exibir os detalhes dos recursos
    print("Recursos presentes nos dados de entrada:")
    print(features_input)
    print("\nRecursos presentes nos dados de treinamento:")
    print(features_training)

    # Exibir qualquer diferença
    if features_only_in_input:
        print("\nRecursos presentes apenas nos dados de entrada:")
        print(features_only_in_input)
    if features_only_in_training:
        print("\nRecursos presentes apenas nos dados de treinamento:")
        print(features_only_in_training)



In [ ]:
# Verificar as características presentes nos dados de entrada e nos dados usados para treinar o escalador
features_scaler = scaler.get_feature_names_out()
features_input_data = eventos_novos_encoded.columns

if set(features_scaler) != set(features_input_data):
    # Se os conjuntos de características forem diferentes, ajuste o escalador novamente
    scaler.fit(eventos_novos_encoded)

# Escalonar os novos dados
eventos_novos_scaled = scaler.transform(eventos_novos_encoded)


# **11. Simulando a aplicação do modelo em dados não vistos**

In [ ]:
# Lista das colunas presentes em X_train, mas ausentes em eventos_novos_encoded
colunas_ausentes = list(set(X_train.columns) - set(eventos_novos_encoded.columns))

# Adicionar as colunas ausentes aos dados de eventos_novos_encoded e preenchê-las com "falso"
for col in colunas_ausentes:
    eventos_novos_encoded[col] = False

# Escalonar os novos dados
eventos_novos_scaled = scaler.transform(eventos_novos_encoded)

# Fazer previsões nos novos dados usando o modelo treinado
predictions = model_pipeline.predict(eventos_novos_scaled)  # Aqui está a correção

# Exibir as previsões
print(predictions)

# Criar um DataFrame com as features de entrada (opcional)
dados_entrada = pd.DataFrame(eventos_novos_encoded, columns=X_train.columns)

# Adicionar uma coluna com os dados preditos
dados_entrada['evento_situacao_num_predito'] = predictions

# Exibir o DataFrame com os dados de entrada e os dados preditos (opcional)
print(dados_entrada)

## 11.1. Fazendo predições em dados novos

In [ ]:
# Fazer previsões nos novos dados usando o modelo treinado
predictions = model_pipeline.predict(eventos_novos_scaled)

# Criar um DataFrame com as features de entrada e as previsões
dados_entrada_predicao = pd.DataFrame(eventos_novos_encoded, columns=X_train.columns)
dados_entrada_predicao['evento_situacao_num_predito'] = predictions

# Mapear as previsões de volta para as classes originais (se necessário)
# predictions_mapped = map_classes(predictions)  # Implemente essa função se aplicável

# Adicionar as previsões mapeadas ao DataFrame
# dados_entrada_predicao['evento_situacao_predito'] = predictions_mapped

# Exibir o DataFrame com os dados de entrada e as previsões
print(dados_entrada_predicao)

## 11.2. Análise de Resultados

In [ ]:
# Contar o número de ocorrências de cada classe prevista
previsoes_counts = dados_entrada_predicao['evento_situacao_num_predito'].value_counts()

# Plotar o gráfico de barras
plt.figure(figsize=(8, 6))
previsoes_counts.plot(kind='bar')
plt.title('Distribuição das Previsões')
plt.xlabel('Classe Prevista')
plt.ylabel('Número de Ocorrências')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Contar o número de ocorrências de cada classe prevista
previsoes_counts = dados_entrada['evento_situacao_num_predito'].value_counts()

# Plotar o gráfico de barras
plt.figure(figsize=(8, 6))
previsoes_counts.plot(kind='bar')
plt.title('Distribuição das Previsões')
plt.xlabel('Classe Prevista')
plt.ylabel('Número de Ocorrências')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Contar o número de ocorrências de cada classe prevista
previsoes_counts = dados_entrada['evento_situacao_num_predito'].value_counts()

# Plotar o gráfico de barras
plt.figure(figsize=(8, 6))
previsoes_counts.plot(kind='bar')
plt.title('Distribuição das Previsões')
plt.xlabel('Classe Prevista')
plt.ylabel('Número de Ocorrências')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Contar o número de ocorrências de cada classe prevista
previsoes_counts = dados_entrada['evento_situacao_num_predito'].value_counts()

# Imprimir os valores totais
print("Total de predições classe 0:", previsoes_counts[0])
print("Total de predições classe 1:", previsoes_counts[1])

**# CHECKLIST AVALIAÇÃO DE RESULTADOS**

**Selecione as métricas de avaliação condizentes com o problema, justificando.**
As métricas selecionadas estão descritas no notebook
Mean Squared Error (MSE - Erro Quadrático Médio):
O MSE é uma medida do erro médio quadrático entre os valores previstos pelo modelo e os valores reais. Quanto menor o valor do MSE, melhor o desempenho do modelo em fazer previsões precisas.

Coeficiente de Determinação (R²):
O R² é uma medida estatística que indica a proporção da variância dos valores dependentes que é explicada pelos valores previstos pelo modelo. Um R² mais próximo de 1 indica um ajuste muito bom do modelo aos dados.

Erro Absoluto Médio (MAE):
O MAE é a média das diferenças absolutas entre os valores previstos pelo modelo e os valores reais. Assim como o MSE, quanto menor o valor do MAE, melhor o desempenho do modelo em fazer previsões precisas.

**Treine o modelo escolhido com toda a base de treino, e teste-o com a base de teste.**
Realizado, está no decorrer do notebook

**Os resultados fazem sentido?**
Sim, confirmou que o modelo está atuando de forma aceitável.

**Foi observado algum problema de overfitting?**
Não

**Compare os resultados de diferentes modelos.**
Modelos comparados no decorrer do notebook

**Descreva a melhor solução encontrada, justificando.**
Árvore de Decisão, justificativa na conclusão.

# **12. Conclusão**

Durante a análise do trabalho, fiquei na dúvida em treinar um modelo de  regressão linear ou de classificação.
A justificativa para regressão linear é que a data hora do evento é algo importante para a análise da situação do evento (Acordado e Não Acordado). E essa informação varia demais, então o ideal é analisar as datas horas futuras juntamente com a sequência de eventos que a unidade geradora possui. Por isso, tinha uma tese que o melhor seria regressão linear.
Por outro lado, o resultado é praticamente dois: Acordado e Não Acordado e isso é mais utilizado para problemas de classificação.
Diante disso, fiz um novo notebook com modelos de classificação. E nesse outro notebook o modelo que teve melhor acurácia também foi a Árvore de Decisão. Com medidas tão altas quanto as de regressão linear.
Por isso resolvi manter esse trabalho utilizando a Regressão Linear. Acredito que não faça diferença já que ela pode ser utilizada tanto para classificação quanto para Regressão.
Mais razões apra utilizar a Árvore de Decisão:



*   Melhor desempenho comparado com outros modelos
*   A aplicação de hiperparâmetros não resultou em melhorias significativas no desempenho do modelo, indicando que o modelo original já estava bem ajustado aos dados.

*   A matriz de confusão mostrou que o modelo teve um desempenho
*   O modelo treinado foi aplicado com sucesso a novos dados, mostrando sua capacidade de generalização.


*   A distribuição das previsões mostra que o modelo classificou os eventos de mudança de estado operativo com precisão, com um número significativo de previsões para cada classe
*   Item da lista


**Justificativa para não utilizar Feature Selection**
Todas as informações relevantes para a análise de consistência do evento foram mantidas no dadaset. Retirei informações que não interessavam para a análise e por isso não achei necessário realizar um tratamento via código da feature selection.
Informações que entendo que são importantes e foram mantidas:

**Usina e Unidade Geradora:** São os agentes proprietários desses equipamentos que determinam a situação do evento. Por isso, foram mantidos, podendo ser determinante para a análise. Por exemplo, o agente X pode sempre contestar um determinado evento.

**Classificadores como evento estado operativo (evento_eo), evento condição operativa (evento_co) e evento origem (evento_or):** Determinam a condição que uma unidade geradora estava. Isso altera a sua disponibilidade e é motivo de análise principal dos agentes.

**Disponibilidade  e data hora:** mesmo caso acima, são pontos importantes para análise dos agentes.

**Conclusão Geral:**
O modelo desenvolvido demonstrou ser eficaz na previsão de contestação de eventos de mudança de estado operativo de unidades geradoras.
No entanto, algumas melhorias precisam ser realizadas, pois proporcionalmente o número de erros se apresenta pequento diante do dataset. É necessário realizar testes com períodos diferentes e colunas novas, não utilizadas no treinamento para que o modelo fique mais eficaz na classificação.